Buoc 1: Khoi tao

In [510]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier


# df = pd.read_csv("IRIS.csv")
# #Lay tat ca du lieu tru cot cuoi
# X = df.iloc[:, :-1].values
# #Lay moi cot cuoi
# y_temp = df.iloc[:, -1].values
# #Cong cu chuyen chu thanh so nguyen
# le = LabelEncoder()
# y = le.fit_transform(y_temp)



Buoc 2: Chia du lieu Train/Test

In [511]:
# X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state = 13)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test  = scaler.transform(X_test)

# 1) load
df = pd.read_csv("KNNAlgorithmDataset.csv")

# 2) chọn cột label (nếu có 'diagnosis' thì dùng nó, nếu không dùng cột cuối)
label_col = 'diagnosis' if 'diagnosis' in df.columns else df.columns[-1]

# 3) loại bỏ cột id nếu có, xóa cột toàn NaN
if 'id' in df.columns: df = df.drop(columns=['id'])
df = df.drop(columns=df.columns[df.isnull().all()])  # drop all-NaN cols

# 4) tách X,y
X = df.drop(columns=[label_col])
y = df[label_col]

# 5) xử lý missing trong X: với numeric -> fill median; categorical -> fill mode + one-hot
X = X.fillna(X.median())               # numeric median
X = pd.get_dummies(X)                  # nếu có categorical -> one-hot

# 6) encode target nếu cần
if y.dtype == 'O' or y.dtype.name == 'category':
    y = LabelEncoder().fit_transform(y)

# 7) split + scale
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.2, random_state=42, shuffle=True)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

Buoc 2: Tinh khoang cach tu z den N diem con lai

In [512]:
def distance(z, X):
    z2 = np.sum(z*z)
    X2 = np.sum(X*X, axis=1)
    return X2 + z2 - 2 * X.dot(z)

Buoc 4: Tim K diem gan nhat

In [513]:
def get_k_nearest_labels(z, X_train, y_train, K):
    dists = distance(z,X_train)
    
    nearist_idx_temp = np.argsort(dists)
    nearist_idx = []
    for i in range(K):
        nearist_idx.append(nearist_idx_temp[i])

    nearest_labels = []
    for i in range (K):
        nearest_labels.append(y_train[nearist_idx[i]])
    
    return nearest_labels

Buoc 5: Gan nhan du lieu

In [514]:
def predict_label(nearest_label):
    x = y = z = 0
    for i in range(len(nearest_label)):
        if nearest_label[i] == 0:
            x += 1
        elif nearest_label[i] == 1:
            y += 1
        else: z += 1
    predict_label = max(x,y,z)
    if predict_label == x : return 0
    elif predict_label == y: return 1
    else: return 2


Buoc 6: Kiem tra voi du lieu thuc te

In [515]:
def evaluate(y_test, X_test, X_train, y_train, K):
    correct = 0
    for i in range(len(X_test)):
        nearest_labels = get_k_nearest_labels(X_test[i], X_train, y_train, K)
        if(y_test[i] == predict_label(nearest_labels)):
            correct += 1
    accuracy = correct / len(X_test)
    return accuracy

In [516]:
K = 12
res = evaluate(y_test, X_test, X_train, y_train, K)
print(f"Độ chính xác: {res*100:.2f}%")

Độ chính xác: 95.61%
